# Example notebook with custom R:R stoploss and takeprofit based on ATR

## With use of SQLite3 datase to create a dataframe



In [4]:
# Load libraries
import pandas as pd
import pandas_ta as pta
import talib.abstract as ta
from pandas import DataFrame
import numpy as np
import mplfinance as mpf
import sqlite3

In [8]:
# Connect to database and create a dataframe from last 999 rows (Descending)
conn = sqlite3.connect('file:../data/BTCUSDT.db?mode=ro', uri=True)
dataframe = pd.read_sql_query("SELECT * FROM _1d ORDER BY openTime DESC LIMIT 999", conn)
conn.close()

# Create human readable time
dataframe['openTime']=(pd.to_datetime(dataframe['openTime'],unit='s'))
# Set openTime to the index, reorder the complete dataframe and add it to a new df for further analysis
df = dataframe.set_index(['openTime']).sort_values(by=['openTime'], ascending=True)

In [9]:
# Show the new df
df

,open,high,low,close,volume,closeTime,quoteAssetVolume,numberOfTrades,takerBuyBaseAssetVolume,takerBuyQuoteAssetVolume,ignore,dow,color,humandate
openTime,,,,,,,,,,,,,,
2019-09-07,10298.71,10558.00,10288.57,10455.88,27637.877392,1567900799,2.878511e+08,272011,14673.489216,1.528259e+08,0,Saturday,Green,20190907
2019-09-08,10455.90,10592.50,10208.00,10381.18,23984.672018,1567987199,2.499010e+08,237752,10588.592458,1.103700e+08,0,Sunday,Red,20190908
2019-09-09,10381.24,10480.00,10068.50,10303.12,39835.727608,1568073599,4.097248e+08,330459,18729.472684,1.928147e+08,0,Monday,Red,20190909
2019-09-10,10302.58,10384.99,9953.00,10098.15,28915.412225,1568159999,2.943238e+08,286763,13432.256822,1.368764e+08,0,Tuesday,Red,20190910
2019-09-11,10098.19,10293.00,9880.00,10158.33,31953.824562,1568246399,3.216371e+08,309120,16583.813097,1.670861e+08,0,Wednesday,Green,20190911
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-28,28629.81,29266.00,28450.00,29031.33,34479.351270,1653782399,9.946743e+08,684423,17561.214650,5.066333e+08,0,Saturday,Green,20220528
2022-05-29,29031.33,29587.78,28839.21,29468.10,27567.347640,1653868799,8.047775e+08,635311,13735.425810,4.010567e+08,0,Sunday,Green,20220529
2022-05-30,29468.10,32222.00,29299.62,31734.22,96785.947600,1653955199,2.980528e+09,1380410,45024.456330,1.384941e+09,0,Monday,Green,20220530


In [12]:
# Show latest entries
df[['open','high','low','close','volume','color']].tail(5)

,open,high,low,close,volume,color
openTime,,,,,,
2022-05-28,28629.81,29266.00,28450.00,29031.33,34479.35127,Green
2022-05-29,29031.33,29587.78,28839.21,29468.10,27567.34764,Green
2022-05-30,29468.10,32222.00,29299.62,31734.22,96785.94760,Green
2022-05-31,31734.23,32399.00,31200.01,31801.04,62433.11632,Green
2022-06-01,31801.05,31982.97,29301.00,29805.83,103395.63382,Red


In [14]:
# Create another dataframe containing only the OHLCV data
df2 = df[['open','high','low','close','volume','color']]
df2

,open,high,low,close,volume,color
openTime,,,,,,
2019-09-07,10298.71,10558.00,10288.57,10455.88,27637.877392,Green
2019-09-08,10455.90,10592.50,10208.00,10381.18,23984.672018,Red
2019-09-09,10381.24,10480.00,10068.50,10303.12,39835.727608,Red
2019-09-10,10302.58,10384.99,9953.00,10098.15,28915.412225,Red
2019-09-11,10098.19,10293.00,9880.00,10158.33,31953.824562,Green
...,...,...,...,...,...,...
2022-05-28,28629.81,29266.00,28450.00,29031.33,34479.351270,Green
2022-05-29,29031.33,29587.78,28839.21,29468.10,27567.347640,Green
2022-05-30,29468.10,32222.00,29299.62,31734.22,96785.947600,Green


## Creating indicators

In [20]:
# MACD indicator
# macd = ta.MACD(
#         df2,
#         fastperiod=12,
#         fastmatype=0,
#         slowperiod=26,
#         slowmatype=0,
#         signalperiod=9,
#         signalmatype=0,
#     )
# df2["macd"] = macd["macd"]
# df2["macdsignal"] = macd["macdsignal"]
# df2["macdhist"] = macd["macdhist"]

# ATR
df2['atr'] = ta.atr(df2, timeperiod=14)

AttributeError: module 'talib.abstract' has no attribute 'atr'